In [21]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import numpy as np

#Cal aerial distance given lat lon

In [2]:
import math

def aerialDist(lat1,lon1,lat2,lon2):

  r = 6371e3
  theta1 = lat1 * math.pi/180
  theta2 = lat2 * math.pi/180

  delta = (lon2 - lon1) * math.pi/180
  d = math.acos(math.sin(theta1) * math.sin(theta2)  + math.cos(theta1) * math.cos(theta2) * math.cos(delta)) * r
  
  return round(d/1000,2)

'''
lat1 = 51.419528
lon1 = -0.220417
lat2 = 51.513592
lon2 = -0.077236
aerialDist(lat1,lon1,lat2,lon2)
'''

'\nlat1 = 51.419528\nlon1 = -0.220417\nlat2 = 51.513592\nlon2 = -0.077236\naerialDist(lat1,lon1,lat2,lon2)\n'

# Total London Check-in (New file)





In [3]:
Total_London_Checkin = pd.read_csv('/content/drive/My Drive/Datathon (1)/Total_London_Checkin.csv',index_col=0)  

In [4]:
def getMainCategory(sub):
  for main, sub_list in heirarchy_cat.items():
    if sub in sub_list or sub == main:
      return main
    elif sub == "Cafe" or sub == "Malaysian Restaurant" or sub == "Ramen /  Noodle House" or sub == "Frozen Yogurt" or sub == "Yogurt":
      return "Food"
    elif sub == "Tennis" or sub == "Athletic & Sport":
      return "Event"
    elif sub == "Ferry" or sub == "Subway" or sub == "Light Rail":
      return "Travel & Transport"
    elif sub == "Car Dealership" or sub == "Drugstore / Pharmacy" or sub == "Mall":
      return "Shop & Service"
    elif sub == "Hiking Trail":
      return "Outdoors & Recreation"
    elif sub == "Spa / Massage" or sub == "Gas Station / Garage":
      return "Shop & Service"
    elif "stadium" in sub.lower() or "pool" in sub.lower() or "scenic lookout" in sub.lower() or "event space" in sub.lower() or "sporting event" in sub.lower():
      return "Event"
  return sub

## Extract category hierarchy

In [5]:
import requests
import json

In [6]:
res = requests.get("https://api.foursquare.com/v2/venues/categories?client_id=HEBZ22QEMCKN3URO3JYLLKB4DKJA211IDVRHPBGKEGJMZG0X&client_secret=LEFXVWWSEJO1UE2ZTMEL5CBQQ5VZOZGWSIJBPHFJ4PTJBGMH&v=20200717").text

In [7]:
res = json.loads(res)

heirarchy_cat = {}

def dfs(heirarchy_cat,name,json_):

  if(json_ == []):
    return

  for j in range(len(json_)):
    heirarchy_cat[name].append(json_[j]['name'])
    dfs(heirarchy_cat,name,json_[j]['categories'])



for i in range(len(res['response']['categories'])):
  json_ = res['response']['categories'][i]['categories']
  name = res['response']['categories'][i]['name']
  heirarchy_cat[name] = []
  dfs(heirarchy_cat,name,json_)

## Get Neighbors Util Function

In [8]:
venue_location_df = Total_London_Checkin[['Venue ID', 'Latitude', 'Longitude', 'Category']].drop_duplicates().reset_index(drop=True)
venue_location_df.loc[venue_location_df['Category']=="Caf\x1a\x1a", "Category"] = "Cafe"
venue_location_df.loc[venue_location_df['Category']=="Caf\x1a", "Category"] = "Cafe"
venue_location_df['MainCategory'] = venue_location_df['Category'].apply(getMainCategory)

In [ ]:
#venue_mainCategory_list = venue_location_df['MainCategory'].unique()

In [9]:
def getVenueNeighbors(venue_id, venue_location_df, radius = 0.5, vtype = None):
  df = venue_location_df[venue_location_df['Venue ID']==venue_id]
  lat_v, lon_v = df['Latitude'].values[0], df['Longitude'].values[0]
  nearbyPlaces = []
  if vtype is not None:
    venuedf = venue_location_df[venue_location_df['Category']==vtype].reset_index(drop=True)
  else: 
    venuedf = venue_location_df
  
  count = 0
  for i in range(len(venuedf)):
    lat_p, lon_p = venuedf['Latitude'].iloc[i], venuedf['Longitude'].iloc[i]
    if lat_p == lat_v and lon_p == lon_v:
      count += 1
      continue
    if (aerialDist(lat_p, lon_p, lat_v, lon_v) <= radius):
      nearbyPlaces.append(venuedf.iloc[i])

  return len(nearbyPlaces) + count, pd.DataFrame(nearbyPlaces).reset_index(drop=True)

# Jensen Coefficient Function

In [ ]:
# subCategoryDist = venue_location_df['Category'].value_counts()
# subCategoryDist

In [ ]:
#np.sum(venue_location_df['Category'].value_counts() > 5)

In [ ]:
# for i in subCategoryDist.index:
#   if "fish" in i.lower():
#     print(i)

In [ ]:
# tp = "Fish & Chips Shop"
# tv = "Wine Shop"

In [10]:
def JensenCoefficient(tp, tv, venue_location_df, venue_id_list):
  N = len(venue_location_df)
  
  N_tp = len(venue_location_df[venue_location_df['Category']==tp])
  N_tv = len(venue_location_df[venue_location_df['Category']==tv])
  #print(N_tp)
  if N_tp == 0:
    N_tp = 1
  
  #print(N_tv)
  if N_tv == 0:
    N_tv = 1

  tot = 0
  for venue_id in venue_id_list:
    nominator = getVenueNeighbors(venue_id=venue_id, venue_location_df = venue_location_df, vtype = tv, radius=0.5)[0]
    denominator = getVenueNeighbors(venue_id=venue_id, venue_location_df = venue_location_df, radius=0.5)[0] - getVenueNeighbors(venue_id=venue_id, venue_location_df = venue_location_df, vtype = tp,radius=0.5)[0]
    if denominator == 0:
      denominator = 1
    term = nominator / denominator
    tot += term
  j_coff = tot * (N - N_tp) / (N_tp * N_tv)

  return j_coff

# London Olympics Check-in (New file)

In [11]:
London_Olympic_Checkin = pd.read_csv('/content/drive/My Drive/Datathon (1)/London_Olympics_Checkin.csv',index_col=0)  
venue_id_list = London_Olympic_Checkin.groupby(['Venue ID']).count()['User ID'].index[(London_Olympic_Checkin.groupby(['Venue ID']).count()['User ID'].values >= 3)]

In [ ]:
#len(venue_id_list)

455

In [12]:
mask = venue_location_df.loc[venue_location_df['Venue ID'].isin(venue_id_list)]['Category'].value_counts().values > 1
subCat_list = venue_location_df.loc[venue_location_df['Venue ID'].isin(venue_id_list)]['Category'].value_counts().index[mask]

In [14]:
len(subCat_list)

66

In [15]:
subCat_list

Index(['Train Station', 'Subway', 'Park', 'Home (private)', 'Hotel',
       'Event Space', 'Pub', 'Stadium', 'Office', 'Neighborhood',
       'General Entertainment', 'Coffee Shop', 'Building', 'Platform',
       'Bridge', 'Plaza', 'Gym', 'Road', 'Grocery Store',
       'Residential Building (Apartment / Condo)', 'Mall',
       'Monument / Landmark', 'Cafe', 'Performing Arts Venue', 'Market',
       'Clothing Store', 'Gay Bar', 'Bar', 'History Museum',
       'Department Store', 'Art Museum', 'Scenic Lookout', 'Flea Market',
       'Art Gallery', 'Soccer Stadium', 'Fast Food Restaurant',
       'General Travel', 'Other Great Outdoors', 'Music Venue', 'Theater',
       'Italian Restaurant', 'Church', 'Bus Line', 'Train', 'Airport Terminal',
       'Racetrack', 'Garden', 'Bus Station', 'Electronics Store',
       'Athletic & Sport', 'Sandwich Place', 'Light Rail', 'Cupcake Shop',
       'Castle', 'Tennis Court', 'Nightclub', 'University',
       'Ramen /  Noodle House', 'Science Museum',

In [13]:
selective = ['Park', 'Home (private)', 'Hotel',
       'Event Space', 'Pub', 'Stadium', 'Office','Coffee Shop','Plaza', 'Gym','Grocery Store',
       'Monument / Landmark','Bar', 'History Museum','Department Store','Scenic Lookout',
       'Soccer Stadium', 'Ramen /  Noodle House','Theater','Church','Farmers Market', 'University']

In [20]:
len(selective)

22

In [ ]:
tp_list = subCat_list
tv_list = subCat_list

In [ ]:
'''
jtable = []
for tp in tp_list:
  for tv in tv_list:
    if tp == tv:
      continue
    else:
      j_coff = JensenCoefficient(tp, tv, venue_location_df, venue_id_list)
      jtable.append([tp, tv, j_coff])
      print("One pair done")

'''

In [ ]:
import time

start_ = time.now()
jtable = []
for tp in tp_list:
  for tv in tv_list:
    if tp == tv:
      continue
    else:
      j_coff = JensenCoefficient(tp, tv, venue_location_df, venue_id_list)
      jtable.append([tp, tv, j_coff])
      print("One pair done")

      found = 1
      break

  if(found == 1):
    break
end = time.now()

print('Time taken is ',(end - start))


In [19]:
j_table = pd.read_csv('/content/drive/My Drive/Datathon (1)/pairs_jcoff_231.txt',delimiter="#",header=None)  
j_table.columns = ["j_coef", 'tp', 'tv']

In [59]:
T = ['Park', 'Home (private)', 'Hotel',
       'Event Space', 'Pub', 'Office','Coffee Shop','Plaza', 'Gym','Grocery Store',
       'Monument / Landmark','Bar', 'History Museum','Department Store','Scenic Lookout',
        'Ramen /  Noodle House','Theater','Church','Farmers Market', 'University']


In [54]:
select_venue_location_df = venue_location_df.loc[venue_location_df['Venue ID'].isin(venue_id_list)].reset_index(drop=True)

In [60]:
venue_temp = select_venue_location_df.loc[select_venue_location_df['Category'].isin(T)].reset_index(drop=True)
venue_temp

,Venue ID,Latitude,Longitude,Category,MainCategory
0,4ac518cef964a520f5a520e3,51.513684,-0.099435,Church,Professional & Other Places
1,4badfc5af964a5202e783be3,51.516835,-0.404166,Coffee Shop,Food
2,4b409606f964a52025b925e3,51.536139,-0.103168,Park,Outdoors & Recreation
3,4f4a8910e4b082dc588778b5,51.431788,0.056610,Grocery Store,Shop & Service
4,4f4ea104e4b0f03ee61495cb,51.431778,0.056673,Grocery Store,Shop & Service
...,...,...,...,...,...
143,501124c3e4b07ac9f07b0dd5,51.547518,-0.023632,Office,Professional & Other Places
144,501e66f8e4b0d9ad7afe6c9a,51.496684,-0.210465,Bar,Nightlife Spot
145,5007ef4fe4b0fa0125d013ec,51.502259,-0.178759,Park,Outdoors & Recreation
146,5010492be4b0871c8a7e25ee,51.535494,-0.105768,Event Space,Event


In [36]:
venue_temp.loc[venue_temp['Category']=='Pub','Venue ID'].values

array(['4aec9f4bf964a52091c921e3', '4ad9f6ddf964a520751c21e3',
       '4ae0acf4f964a5208c8121e3', ..., '4df7b9612271d8baf9c4c600',
       '500abac5e4b0510620e25b28', '4ba0e8f6f964a520608637e3'],
      dtype=object)

In [43]:
from tqdm import tqdm

In [73]:
def calculate_N_tp_bar(tp, tv):
  all_tv = select_venue_location_df.loc[select_venue_location_df['Category']==tv,'Venue ID'].values
  tot = 0
  for v_id in all_tv:
    N_tp = getVenueNeighbors(v_id, venue_location_df, radius = 0.5, vtype = tp)[0]
    tot += N_tp
  return tot/len(all_tv)

In [74]:
def JensenQuality(venue_id):
  #print(venue_id)
  tv = venue_temp.loc[venue_temp['Venue ID']==venue_id,'Category'].values[0] #need to change venue_temp
  tot = 0
  for tp in tqdm(T):
    if tp == tv:
      continue
    # print("tv: "+tv)
    # print("tp: "+tp)
    try:
      k_tp_tv = j_table.loc[(j_table['tp'] == tp)&(j_table['tv'] == tv),'j_coef'].values[0]
    except:
      k_tp_tv = j_table.loc[(j_table['tp'] == tv)&(j_table['tv'] == tp),'j_coef'].values[0]
    N_tp = getVenueNeighbors(venue_id, venue_location_df, radius = 0.5, vtype = tp)[0]
    N_tp_bar = calculate_N_tp_bar(tp, tv)
    term = k_tp_tv * (N_tp - N_tp_bar)
    tot += term
  print(tot)
  return tot

In [75]:
venue_temp['JensenQ'] = venue_temp['Venue ID'].apply(JensenQuality)




  0%|          | 0/20 [00:00<?, ?it/s]


 10%|█         | 2/20 [00:00<00:01, 10.17it/s]


 15%|█▌        | 3/20 [00:00<00:01,  9.58it/s]


 25%|██▌       | 5/20 [00:00<00:01,  8.65it/s]


 30%|███       | 6/20 [00:00<00:02,  6.65it/s]


 35%|███▌      | 7/20 [00:00<00:01,  6.80it/s]


 45%|████▌     | 9/20 [00:01<00:01,  8.25it/s]


 50%|█████     | 10/20 [00:01<00:01,  8.31it/s]


 60%|██████    | 12/20 [00:01<00:00,  8.99it/s]


 70%|███████   | 14/20 [00:01<00:00, 10.67it/s]


100%|██████████| 20/20 [00:01<00:00, 11.88it/s]



  0%|          | 0/20 [00:00<?, ?it/s]

38.201100698593734





  5%|▌         | 1/20 [00:00<00:04,  4.62it/s]


 10%|█         | 2/20 [00:00<00:04,  4.04it/s]


 15%|█▌        | 3/20 [00:00<00:04,  3.66it/s]


 20%|██        | 4/20 [00:01<00:03,  4.19it/s]


 25%|██▌       | 5/20 [00:01<00:05,  2.81it/s]


 30%|███       | 6/20 [00:02<00:06,  2.33it/s]


 40%|████      | 8/20 [00:02<00:03,  3.15it/s]


 45%|████▌     | 9/20 [00:02<00:02,  3.70it/s]


 50%|█████     | 10/20 [00:02<00:02,  3.65it/s]


 60%|██████    | 12/20 [00:03<00:02,  3.90it/s]


 70%|███████   | 14/20 [00:03<00:01,  4.79it/s]


 80%|████████  | 16/20 [00:03<00:00,  5.83it/s]


 85%|████████▌ | 17/20 [00:03<00:00,  6.30it/s]


 90%|█████████ | 18/20 [00:03<00:00,  6.59it/s]


100%|██████████| 20/20 [00:04<00:00,  4.93it/s]



  0%|          | 0/20 [00:00<?, ?it/s]

-67.41634653774297





 10%|█         | 2/20 [00:00<00:04,  4.02it/s]


 15%|█▌        | 3/20 [00:00<00:05,  3.13it/s]


 20%|██        | 4/20 [00:01<00:04,  3.40it/s]


 25%|██▌       | 5/20 [00:02<00:07,  2.11it/s]


 30%|███       | 6/20 [00:02<00:07,  1.75it/s]


 35%|███▌      | 7/20 [00:03<00:07,  1.79it/s]


 40%|████      | 8/20 [00:03<00:05,  2.27it/s]


 45%|████▌     | 9/20 [00:03<00:04,  2.64it/s]


 50%|█████     | 10/20 [00:04<00:03,  2.56it/s]


 55%|█████▌    | 11/20 [00:04<00:02,  3.16it/s]


 60%|██████    | 12/20 [00:04<00:02,  2.81it/s]


 65%|██████▌   | 13/20 [00:04<00:02,  3.47it/s]


 70%|███████   | 14/20 [00:05<00:01,  4.01it/s]


 75%|███████▌  | 15/20 [00:05<00:01,  4.67it/s]


 80%|████████  | 16/20 [00:05<00:00,  5.36it/s]


 85%|████████▌ | 17/20 [00:05<00:00,  5.46it/s]


 90%|█████████ | 18/20 [00:05<00:00,  5.29it/s]


 95%|█████████▌| 19/20 [00:05<00:00,  5.91it/s]


100%|██████████| 20/20 [00:06<00:00,  3.30it/s]



  0%|          | 0/20 [00:00<?, ?it/s]


  5%|▌       

...

-4.924693785677902





  5%|▌         | 1/20 [00:00<00:04,  3.83it/s]


 10%|█         | 2/20 [00:00<00:05,  3.30it/s]


 15%|█▌        | 3/20 [00:01<00:05,  3.02it/s]


 20%|██        | 4/20 [00:01<00:04,  3.44it/s]


 25%|██▌       | 5/20 [00:02<00:06,  2.30it/s]


 35%|███▌      | 7/20 [00:02<00:04,  2.70it/s]


 40%|████      | 8/20 [00:02<00:03,  3.31it/s]


 45%|████▌     | 9/20 [00:02<00:02,  3.68it/s]


 50%|█████     | 10/20 [00:03<00:02,  3.41it/s]


 55%|█████▌    | 11/20 [00:03<00:02,  4.15it/s]


 60%|██████    | 12/20 [00:03<00:02,  3.53it/s]


 65%|██████▌   | 13/20 [00:03<00:01,  4.28it/s]


 70%|███████   | 14/20 [00:03<00:01,  4.93it/s]


 75%|███████▌  | 15/20 [00:04<00:00,  5.77it/s]


 80%|████████  | 16/20 [00:04<00:00,  6.48it/s]


 85%|████████▌ | 17/20 [00:04<00:00,  6.73it/s]


 90%|█████████ | 18/20 [00:04<00:00,  6.41it/s]


 95%|█████████▌| 19/20 [00:04<00:00,  7.01it/s]


100%|██████████| 20/20 [00:04<00:00,  4.29it/s]



  0%|          | 0/20 [00:00<?, ?it/s]

-6.753456592606314





  5%|▌         | 1/20 [00:00<00:05,  3.75it/s]


 10%|█         | 2/20 [00:00<00:05,  3.28it/s]


 15%|█▌        | 3/20 [00:01<00:05,  3.01it/s]


 20%|██        | 4/20 [00:01<00:04,  3.43it/s]


 25%|██▌       | 5/20 [00:01<00:06,  2.36it/s]


 35%|███▌      | 7/20 [00:02<00:04,  2.75it/s]


 40%|████      | 8/20 [00:02<00:03,  3.36it/s]


 45%|████▌     | 9/20 [00:02<00:02,  3.74it/s]

KeyboardInterrupt: ignored